In [1290]:
import pandas as pd
import locale
import pickle
locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [1291]:
with open('../src/depara/trends/categoria.pkl', 'rb') as arq:
    lista_categoria = pickle.load(arq)
lista_categoria

ID        NOME_CATEGORIA
0    1      Film & Animation
1    2      Autos & Vehicles
2   10                 Music
3   15        Pets & Animals
4   17                Sports
5   18          Short Movies
6   19       Travel & Events
7   20                Gaming
8   21         Videoblogging
9   22        People & Blogs
10  23                Comedy
11  24         Entertainment
12  25       News & Politics
13  26         Howto & Style
14  27             Education
15  28  Science & Technology
16  30                Movies
17  31       Anime/Animation
18  32      Action/Adventure
19  33              Classics
20  34                Comedy
21  35           Documentary
22  36                 Drama
23  37                Family
24  38               Foreign
25  39                Horror
26  40        Sci-Fi/Fantasy
27  41              Thriller
28  42                Shorts
29  43                 Shows
30  44              Trailers

In [1292]:
dtypes = {
    'ID_CATEGORIA' : 'int32',
    'ID_CANAL' : 'string',
    'NM_CANAL' : 'string',
    'ID_VIDEO' : 'string',
    'TITULO_VIDEO' : 'string',
    'DURACAO' : 'string',
    'DESCRICAO': 'string',
    'TOTAL_COMENTARIOS' : 'float32',
    'TOTAL_FAVORITOS': 'int32',
    'TOTAL_LIKES': 'float32',
    'TOTAL_VISUALIZACOES': 'int32'
}

# Funções gerais

In [1322]:
def calcular_variacoes_gerais(dataframe: pd.DataFrame) -> pd.DataFrame:
    dataframe['TOTAL_VISUALIZACOES_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_VISUALIZACOES'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_VISUALIZACOES_DIA_ATUAL'] = dataframe['TOTAL_VISUALIZACOES'] - dataframe['TOTAL_VISUALIZACOES_DESLOCADO']

    dataframe['TOTAL_COMENTARIOS_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_COMENTARIOS'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_COMENTARIOS_DIA_ATUAL'] = dataframe['TOTAL_COMENTARIOS'] - dataframe['TOTAL_COMENTARIOS_DESLOCADO']
    
    dataframe['TOTAL_LIKES_DESLOCADO'] = dataframe.groupby(['ID_VIDEO'])['TOTAL_LIKES'].shift(1)
    dataframe.fillna(0, axis=1, inplace=True)
    dataframe['TOTAL_LIKES_DIA_ATUAL'] = dataframe['TOTAL_LIKES'] - dataframe['TOTAL_LIKES_DESLOCADO']
    dataframe.drop(['TOTAL_COMENTARIOS_DESLOCADO', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES_DESLOCADO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES_DESLOCADO', 'TOTAL_LIKES'], axis=1, inplace=True)

    return dataframe

In [1294]:
def traduzir_dia_da_semana(dia):
    traducao = {
        'Sunday': 'Domingo',
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado'
    }
    return traducao.get(dia, dia)

In [1295]:
def turno_semana(hora: int):
    if 0 <= hora < 6:
        turno = 'Madrugada', 1
    elif 6 <= hora < 12:
        turno = 'Manhã' , 2
    elif 12 <= hora < 18:
        turno = 'Tarde', 3
    else:
        turno = 'Noite', 4
    return turno

# Abrir dataframe

In [1296]:
datas = [
    'extracao_data_2023_10_15',
    'extracao_data_2023_10_16', 
    'extracao_data_2023_10_17', 
    'extracao_data_2023_10_18',
    'extracao_data_2023_10_19',
    'extracao_data_2023_10_20',
    'extracao_data_2023_10_21',
    'extracao_data_2023_10_22',
    'extracao_data_2023_10_23',
    'extracao_data_2023_10_24',
    'extracao_data_2023_10_25',
    'extracao_data_2023_10_26',
    'extracao_data_2023_10_27']
base_original = pd.DataFrame()
for path in datas:
    try:
        base_temp = pd.read_parquet(f'../data/prata/top_brazil/{path}/historico_top_brazil.parquet')
        base_original = pd.concat([base_original, base_temp])
    except Exception as e:
        print(path)
        print()
        pass

extracao_data_2023_10_17

extracao_data_2023_10_18



In [1297]:
base_original.head()

data_extracao ID_CATEGORIA                  ID_CANAL     NM_CANAL  \
0  2023-10-15 14:38:56           24  UCX6OQ3DkcsbYNE6H8uQQuVA      MrBeast   
1  2023-10-15 14:38:56           25  UC376n347Ob5Lwzq2WGzF1AA     SBT News   
2  2023-10-15 14:38:56           20  UCmhx7Q1WRB85e7bF-MHcehA       Ligonz   
3  2023-10-15 14:38:56           20  UCcqcrpCct3dlF8H-3lv7sKA         Nait   
4  2023-10-15 14:38:56           10  UCRD1ypvmK7e_05Rtu9zHOfQ  Ana Castela   

      ID_VIDEO                                       TITULO_VIDEO   DURACAO  \
0  3ryID_SwU5E                          $1 Vs $100,000,000 House!  PT17M36S   
1  146wNN1qwPk  Terminou prazo de Israel para que palestinos d...   PT8M41S   
2  uqN9GrkAxOI                    FUI EXPULSO DA CREATIVE SQUAD 4     PT15M   
3  RUNExoWpWbc      24 HORAS para ser DESBANIDO da Creative Squad  PT11M55S   
4  IgG1ekanyFQ  Ana Castela - Alerta de Golpe (DVD Boiadeira I...   PT2M30S   

                                           DESCRICAO TOTAL_COMENTARIOS  \
0  I can’t believe how expensive the last house i...            100754   
1  Terminou, no início desta noite (13.out), o pr...              2683   
2  #creativesquad4 #cs4  #expulso #genshinimpact ...               335   
3  BAIXE AGORA GENSHIN IMPACT E VENHA CONHECER ES...               220   
4  Ouça agora: https://agroplaybr.ffm.to/boiadeir...              1789   

  TOTAL_FAVORITOS TOTAL_LIKES TOTAL_VISUALIZACOES  
0               0     3182804            64102803  
1               0       19545             1050551  
2               0       11423              239520  
3               0        8113              138405  
4               0       29201              931886

In [1298]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   data_extracao        29399 non-null  object
 1   ID_CATEGORIA         29399 non-null  object
 2   ID_CANAL             29399 non-null  object
 3   NM_CANAL             29399 non-null  object
 4   ID_VIDEO             29399 non-null  object
 5   TITULO_VIDEO         29399 non-null  object
 6   DURACAO              29399 non-null  object
 7   DESCRICAO            29399 non-null  object
 8   TOTAL_COMENTARIOS    28472 non-null  object
 9   TOTAL_FAVORITOS      29399 non-null  object
 10  TOTAL_LIKES          28796 non-null  object
 11  TOTAL_VISUALIZACOES  29399 non-null  object
dtypes: object(12)
memory usage: 2.9+ MB


In [1299]:
base_original['data_extracao'] = base_original['data_extracao'].astype('datetime64[ns]')
base_original['ID_CATEGORIA'] = base_original['ID_CATEGORIA'].astype('string')
base_original['ID_CANAL'] = base_original['ID_CANAL'].astype('string')
base_original['NM_CANAL'] = base_original['NM_CANAL'].astype('string')
base_original['ID_VIDEO'] = base_original['ID_VIDEO'].astype('string')
base_original['TITULO_VIDEO'] = base_original['TITULO_VIDEO'].astype('string')
base_original['DURACAO'] = base_original['DURACAO'].astype('string')
base_original['DESCRICAO'] = base_original['DESCRICAO'].astype('string')
base_original['TOTAL_COMENTARIOS'] = base_original['TOTAL_COMENTARIOS'].astype('float32')
base_original['TOTAL_FAVORITOS'] = base_original['TOTAL_FAVORITOS'].astype('float32')
base_original['TOTAL_LIKES'] = base_original['TOTAL_LIKES'].astype('float32')
base_original['TOTAL_VISUALIZACOES'] = base_original['TOTAL_VISUALIZACOES'].astype('float32')


In [1300]:
base_original['TOTAL_FAVORITOS'] = base_original['TOTAL_FAVORITOS'].astype('int32',errors='ignore')
base_original['TOTAL_LIKES'] = base_original['TOTAL_LIKES'].astype('int32', errors='ignore')
base_original['TOTAL_VISUALIZACOES'] = base_original['TOTAL_VISUALIZACOES'].astype('int32', errors='ignore')


In [1301]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   data_extracao        29399 non-null  datetime64[ns]
 1   ID_CATEGORIA         29399 non-null  string        
 2   ID_CANAL             29399 non-null  string        
 3   NM_CANAL             29399 non-null  string        
 4   ID_VIDEO             29399 non-null  string        
 5   TITULO_VIDEO         29399 non-null  string        
 6   DURACAO              29399 non-null  string        
 7   DESCRICAO            29399 non-null  string        
 8   TOTAL_COMENTARIOS    28472 non-null  float32       
 9   TOTAL_FAVORITOS      29399 non-null  int32         
 10  TOTAL_LIKES          28796 non-null  float32       
 11  TOTAL_VISUALIZACOES  29399 non-null  int32         
dtypes: datetime64[ns](1), float32(2), int32(2), string(7)
memory usage: 2.5 MB


In [1302]:
base_original['DATA_EXTRACAO_API'] = base_original['data_extracao']
base_original.drop('data_extracao', axis=1, inplace=True)

In [1303]:
base_original['DATA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.date.astype('datetime64[ns]')
base_original['HORA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.hour
base_original['TURNO_EXTRACAO'] = base_original['HORA_EXTRACAO'].apply(lambda x : turno_semana(x)[0]).astype('string')
base_original['SEMANA_EXTRACAO'] = base_original['DATA_EXTRACAO_API'].dt.strftime('%A').str.capitalize().astype('string')
base_original['INDICE_TURNO_EXTRACAO'] =  base_original['HORA_EXTRACAO'].apply(lambda x : turno_semana(x)[1]).astype('int32')
base_original['INDICE_SEMANA_EXTRACAO'] = base_original['DATA_EXTRACAO'].dt.day_of_week
base_original

ID_CATEGORIA                  ID_CANAL       NM_CANAL     ID_VIDEO  \
0              24  UCX6OQ3DkcsbYNE6H8uQQuVA        MrBeast  3ryID_SwU5E   
1              25  UC376n347Ob5Lwzq2WGzF1AA       SBT News  146wNN1qwPk   
2              20  UCmhx7Q1WRB85e7bF-MHcehA         Ligonz  uqN9GrkAxOI   
3              20  UCcqcrpCct3dlF8H-3lv7sKA           Nait  RUNExoWpWbc   
4              10  UCRD1ypvmK7e_05Rtu9zHOfQ    Ana Castela  IgG1ekanyFQ   
...           ...                       ...            ...          ...   
2345           20  UCKAdZfnAhMY9RFSD4qL4KCQ       Tigre TV  EV8_D0Mx6aI   
2346           10  UCDVAzIDnbgElNB4XCrYLvsA   Bruno Gadiol  fDbNLQSddTQ   
2347           10  UCfUEYnV7UjMRfBGY0bfEkPA         JOTAPÊ  Ol_t-hgXrv0   
2348           10  UCFZsvdD_Sb98qFdtSU7Et2Q  Tz da Coronel  XyA8jDHTEcU   
2349           20  UCtXXSuHUWh1-PRC7AeDB7mg        Saadhak  f9TbPK2KVrY   

                                           TITULO_VIDEO   DURACAO  \
0                             $1 Vs $100,000,000 House!  PT17M36S   
1     Terminou prazo de Israel para que palestinos d...   PT8M41S   
2                       FUI EXPULSO DA CREATIVE SQUAD 4     PT15M   
3         24 HORAS para ser DESBANIDO da Creative Squad  PT11M55S   
4     Ana Castela - Alerta de Golpe (DVD Boiadeira I...   PT2M30S   
...                                                 ...       ...   
2345  O DIA CHEGOU!!! ATUALIZAÇÃO 20 do BLOX FRUITS ...  PT12M43S   
2346  Bruno Gadiol, Grag Queen - f:)da-se (Videoclip...   PT2M55S   
2347  JOTAPÊ - JARDIM DAS FLORES (Prod. Marquinho no...   PT3M23S   
2348  Tz da Coronel - Castelo de Areia feat. Veigh &...   PT3M43S   
2349  TUDO sobre ISO NOVO AGENTE  do VALORANT | LOUD...  PT13M30S   

                                              DESCRICAO  TOTAL_COMENTARIOS  \
0     I can’t believe how expensive the last house i...           100754.0   
1     Terminou, no início desta noite (13.out), o pr...             2683.0   
2     #creativesquad4 #cs4  #expulso #genshinimpact ...              335.0   
3     BAIXE AGORA GENSHIN IMPACT E VENHA CONHECER ES...              220.0   
4     Ouça agora: https://agroplaybr.ffm.to/boiadeir...             1789.0   
...                                                 ...                ...   
2345  Use StarCode ⇢ TigreTV ⇠ e ajude o tigrinho he...              502.0   
2346  Esse é o videoclipe oficial de "f:)da-se".

Cl...              362.0   
2347  Ouça #jardimdasflores em todas as plataformas ...             4034.0   
2348  Ouça o álbum #DACOROMODE em todas as plataform...             1271.0   
2349  Matias Delipetro - Saadhak 🐦 📷
. Twitter https...              568.0   

      TOTAL_FAVORITOS  TOTAL_LIKES  TOTAL_VISUALIZACOES   DATA_EXTRACAO_API  \
0                   0    3182804.0             64102804 2023-10-15 14:38:56   
1                   0      19545.0              1050551 2023-10-15 14:38:56   
2                   0      11423.0               239520 2023-10-15 14:38:56   
3                   0       8113.0               138405 2023-10-15 14:38:56   
4                   0      29201.0               931886 2023-10-15 14:38:56   
...               ...          ...                  ...                 ...   
2345                0       4833.0               101571 2023-10-27 19:02:33   
2346                0       7590.0                62678 2023-10-27 19:02:33   
2347                0      66104.0               538508 2023-10-27 19:02:33   
2348                0      46277.0              1003517 2023-10-27 19:02:33   
2349                0      16357.0               205837 2023-10-27 19:02:33   

     DATA_EXTRACAO  HORA_EXTRACAO TURNO_EXTRACAO SEMANA_EXTRACAO  \
0       2023-10-15             14          Tarde         Domingo   
1       2023-10-15             14          Tarde         Domingo   
2       2023-10-15             14          Tarde         Domingo   
3       2023-10-15             14          Tarde         Domingo   
4       2023-10-15             14          Tarde 

# popularidade categoria

- Valor max por turno

In [1304]:
base_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29399 entries, 0 to 2349
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_CATEGORIA            29399 non-null  string        
 1   ID_CANAL                29399 non-null  string        
 2   NM_CANAL                29399 non-null  string        
 3   ID_VIDEO                29399 non-null  string        
 4   TITULO_VIDEO            29399 non-null  string        
 5   DURACAO                 29399 non-null  string        
 6   DESCRICAO               29399 non-null  string        
 7   TOTAL_COMENTARIOS       28472 non-null  float32       
 8   TOTAL_FAVORITOS         29399 non-null  int32         
 9   TOTAL_LIKES             28796 non-null  float32       
 10  TOTAL_VISUALIZACOES     29399 non-null  int32         
 11  DATA_EXTRACAO_API       29399 non-null  datetime64[ns]
 12  DATA_EXTRACAO           29399 non-null  datetime64[n

In [1305]:
base_original['DATA_EXTRACAO'].unique()

<DatetimeArray>
['2023-10-15 00:00:00', '2023-10-16 00:00:00', '2023-10-19 00:00:00',
 '2023-10-20 00:00:00', '2023-10-21 00:00:00', '2023-10-22 00:00:00',
 '2023-10-23 00:00:00', '2023-10-24 00:00:00', '2023-10-25 00:00:00',
 '2023-10-26 00:00:00', '2023-10-27 00:00:00']
Length: 11, dtype: datetime64[ns]

In [1306]:
base_original = base_original.sort_values(by=['DATA_EXTRACAO', 'INDICE_TURNO_EXTRACAO', 'ID_CATEGORIA', 'ID_CANAL', 'ID_VIDEO'])


In [1307]:
base_max = base_original[['DATA_EXTRACAO', 'TURNO_EXTRACAO','INDICE_TURNO_EXTRACAO', 'ID_CATEGORIA', 'ID_CANAL', 'ID_VIDEO', 'TOTAL_VISUALIZACOES']]
base_max.query('ID_VIDEO == "8KVsaoveTbw" and DATA_EXTRACAO == "2023-10-15" ').sort_values(by=['DATA_EXTRACAO', 'INDICE_TURNO_EXTRACAO'])

DATA_EXTRACAO TURNO_EXTRACAO  INDICE_TURNO_EXTRACAO ID_CATEGORIA  \
64      2023-10-15          Tarde                      3            1   
264     2023-10-15          Tarde                      3            1   
464     2023-10-15          Tarde                      3            1   
664     2023-10-15          Tarde                      3            1   
864     2023-10-15          Tarde                      3            1   
1064    2023-10-15          Tarde                      3            1   
1264    2023-10-15          Tarde                      3            1   
1463    2023-10-15          Tarde                      3            1   
1695    2023-10-15          Tarde                      3            1   
1894    2023-10-15          Noite                      4            1   
2094    2023-10-15          Noite                      4            1   
2294    2023-10-15          Noite                      4            1   
2494    2023-10-15          Noite                      4            1   
2694    2023-10-15          Noite                      4            1   
2894    2023-10-15          Noite                      4            1   
3094    2023-10-15          Noite                      4            1   
3294    2023-10-15          Noite                      4            1   

                      ID_CANAL     ID_VIDEO  TOTAL_VISUALIZACOES  
64    UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4029760  
264   UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4030703  
464   UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4033786  
664   UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4042055  
864   UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4042055  
1064  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4058589  
1264  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4076406  
1463  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4094821  
1695  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4145625  
1894  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4164634  
2094  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4189988  
2294  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4189988  
2494  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4189988  
2694  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4204161  
2894  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4224058  
3094  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4255404  
3294  UCuPivVjnfNo4mb3Oog_frZg  8KVsaoveTbw              4279413

In [1325]:
base_max = base_original.groupby(['DATA_EXTRACAO' ,'ID_CATEGORIA', 'ID_VIDEO']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'max'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS', 'max'),
    TOTAL_LIKES=('TOTAL_LIKES', 'max'),

).reset_index().sort_values(by=['ID_CATEGORIA', 'ID_VIDEO', 'DATA_EXTRACAO'])
base_max

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO  TOTAL_VISUALIZACOES  \
0       2023-10-15            1  8KVsaoveTbw              4279413   
231     2023-10-16            1  8KVsaoveTbw              4695234   
471     2023-10-19            1  8KVsaoveTbw              5407456   
1       2023-10-15            1  CYcrmsdZuyw              4141018   
232     2023-10-16            1  CYcrmsdZuyw              4355662   
...            ...          ...          ...                  ...   
230     2023-10-15           28  v58gkYSVZR8               145976   
470     2023-10-16           28  v58gkYSVZR8               147478   
699     2023-10-19           28  v58gkYSVZR8               151752   
920     2023-10-20           28  v58gkYSVZR8               153609   
1134    2023-10-21           28  v58gkYSVZR8               153788   

      TOTAL_COMENTARIOS  TOTAL_LIKES  
0                5415.0      78174.0  
231              5706.0      83626.0  
471              6045.0      89762.0  
1               17758.0     171952.0  
232             18443.0     181368.0  
...                 ...          ...  
230               236.0       2956.0  
470               233.0       2970.0  
699               232.0       2984.0  
920               232.0       2985.0  
1134              232.0       2984.0  

[2547 rows x 6 columns]

In [1326]:
base_max = calcular_variacoes_gerais(base_max)
base_max

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO  \
0     2023-10-15 00:00:00            1  8KVsaoveTbw   
231   2023-10-16 00:00:00            1  8KVsaoveTbw   
471   2023-10-19 00:00:00            1  8KVsaoveTbw   
1     2023-10-15 00:00:00            1  CYcrmsdZuyw   
232   2023-10-16 00:00:00            1  CYcrmsdZuyw   
...                   ...          ...          ...   
230   2023-10-15 00:00:00           28  v58gkYSVZR8   
470   2023-10-16 00:00:00           28  v58gkYSVZR8   
699   2023-10-19 00:00:00           28  v58gkYSVZR8   
920   2023-10-20 00:00:00           28  v58gkYSVZR8   
1134  2023-10-21 00:00:00           28  v58gkYSVZR8   

     TOTAL_VISUALIZACOES_DIA_ATUAL TOTAL_COMENTARIOS_DIA_ATUAL  \
0                          4279413                      5415.0   
231                       415821.0                       291.0   
471                       712222.0                       339.0   
1                          4141018                     17758.0   
232                       214644.0                       685.0   
...                            ...                         ...   
230                         145976                       236.0   
470                         1502.0                        -3.0   
699                         4274.0                        -1.0   
920                         1857.0                         0.0   
1134                         179.0                         0.0   

     TOTAL_LIKES_DIA_ATUAL  
0                  78174.0  
231                 5452.0  
471                 6136.0  
1                 171952.0  
232                 9416.0  
...                    ...  
230                 2956.0  
470                   14.0  
699                   14.0  
920                    1.0  
1134                  -1.0  

[2547 rows x 6 columns]

In [1329]:
base_max.head(10)

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO  \
0    2023-10-15 00:00:00            1  8KVsaoveTbw   
231  2023-10-16 00:00:00            1  8KVsaoveTbw   
471  2023-10-19 00:00:00            1  8KVsaoveTbw   
1    2023-10-15 00:00:00            1  CYcrmsdZuyw   
232  2023-10-16 00:00:00            1  CYcrmsdZuyw   
472  2023-10-19 00:00:00            1  CYcrmsdZuyw   
700  2023-10-20 00:00:00            1  CYcrmsdZuyw   
921  2023-10-21 00:00:00            1  CYcrmsdZuyw   
2    2023-10-15 00:00:00            1  KKB5C1Dun-Q   
233  2023-10-16 00:00:00            1  KKB5C1Dun-Q   

    TOTAL_VISUALIZACOES_DIA_ATUAL TOTAL_COMENTARIOS_DIA_ATUAL  \
0                         4279413                      5415.0   
231                      415821.0                       291.0   
471                      712222.0                       339.0   
1                         4141018                     17758.0   
232                      214644.0                       685.0   
472                      528646.0                      1004.0   
700                      100918.0                       295.0   
921                       95882.0                       158.0   
2                          269891                       114.0   
233                        1771.0                         1.0   

    TOTAL_LIKES_DIA_ATUAL  
0                 78174.0  
231                5452.0  
471                6136.0  
1                171952.0  
232                9416.0  
472               18951.0  
700                3290.0  
921                2492.0  
2                  3684.0  
233                  19.0

In [1339]:
base_soma = base_max.groupby(['DATA_EXTRACAO', 'ID_CATEGORIA']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES_DIA_ATUAL', 'sum'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS_DIA_ATUAL', 'sum'),
    TOTAL_LIKES=('TOTAL_LIKES_DIA_ATUAL', 'sum'),
).reset_index()

base_soma[['TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES']] = base_soma[['TOTAL_VISUALIZACOES', 'TOTAL_COMENTARIOS', 'TOTAL_LIKES']].astype('int')
base_soma.sort_values(by=['TOTAL_VISUALIZACOES'], ascending=False)

DATA_EXTRACAO ID_CATEGORIA  TOTAL_VISUALIZACOES  TOTAL_COMENTARIOS  \
6     2023-10-15           24            285267739             572539   
1     2023-10-15           10            188470119             505272   
31    2023-10-19           24             51989464             100616   
44    2023-10-20           24             30917825             133974   
3     2023-10-15           20             29314189             116548   
..           ...          ...                  ...                ...   
40    2023-10-20            2                 6535                 58   
78    2023-10-23            2                 6156                 19   
33    2023-10-19           26                 2742                 10   
20    2023-10-16           26                 2227                 10   
62    2023-10-22            1                  142                  1   

    TOTAL_LIKES  
6      11743140  
1       8054410  
31      2219157  
44       882030  
3       1747021  
..          ...  
40          423  
78          366  
33          224  
20          199  
62            0  

[135 rows x 5 columns]

In [1340]:
dataframe = pd.merge(
   lista_categoria, base_soma, how='inner', left_on=['ID'], right_on=['ID_CATEGORIA'])

In [1341]:
dataframe = dataframe.sort_values(by=['DATA_EXTRACAO'])

In [1342]:
dataframe 


ID        NOME_CATEGORIA DATA_EXTRACAO ID_CATEGORIA  TOTAL_VISUALIZACOES  \
0     1      Film & Animation    2023-10-15            1             10646952   
102  26         Howto & Style    2023-10-15           26                49953   
91   25       News & Politics    2023-10-15           25             11520764   
80   24         Entertainment    2023-10-15           24            285267739   
69   23                Comedy    2023-10-15           23              1361139   
..   ..                   ...           ...          ...                  ...   
79   23                Comedy    2023-10-27           23                93108   
90   24         Entertainment    2023-10-27           24             27166750   
101  25       News & Politics    2023-10-27           25               115333   
46   19       Travel & Events    2023-10-27           19                46462   
134  28  Science & Technology    2023-10-27           28               125570   

     TOTAL_COMENTARIOS  TOTAL_LIKES  
0                36792       487262  
102                690         9076  
91               13235       248596  
80              572539     11743140  
69                1486        68949  
..                 ...          ...  
79                 108         3210  
90               74545      1231578  
101                831         4246  
46                 280         3604  
134               1870        11246  

[135 rows x 7 columns]

In [1343]:
dataframe.drop(columns=['ID'], axis=1, inplace=True)
dataframe['NOME_CATEGORIA'] = dataframe['NOME_CATEGORIA'].astype('string')

In [1344]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135 entries, 0 to 134
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   NOME_CATEGORIA       135 non-null    string        
 1   DATA_EXTRACAO        135 non-null    datetime64[ns]
 2   ID_CATEGORIA         135 non-null    object        
 3   TOTAL_VISUALIZACOES  135 non-null    int32         
 4   TOTAL_COMENTARIOS    135 non-null    int32         
 5   TOTAL_LIKES          135 non-null    int32         
dtypes: datetime64[ns](1), int32(3), object(1), string(1)
memory usage: 5.8+ KB


In [1345]:
dataframe.to_parquet('../data/ouro_csv/trends/popularidade_categoria_trends.parquet')

- Vídeo populares por categoria (Top 10) (likes, comentarios e visualizações) Selecionado por dia

In [1260]:
base_original.columns

Index(['ID_CATEGORIA', 'ID_CANAL', 'NM_CANAL', 'ID_VIDEO', 'TITULO_VIDEO',
       'DURACAO', 'DESCRICAO', 'TOTAL_COMENTARIOS', 'TOTAL_FAVORITOS',
       'TOTAL_LIKES', 'TOTAL_VISUALIZACOES', 'DATA_EXTRACAO_API',
       'DATA_EXTRACAO', 'HORA_EXTRACAO', 'TURNO_EXTRACAO', 'SEMANA_EXTRACAO',
       'INDICE_TURNO_EXTRACAO', 'INDICE_SEMANA_EXTRACAO'],
      dtype='object')

In [1261]:
df_videos_populares_categoria = base_original[
    ['DATA_EXTRACAO', 'ID_CATEGORIA',  'ID_VIDEO', 'TITULO_VIDEO', 'TOTAL_COMENTARIOS', 'TOTAL_FAVORITOS','TOTAL_LIKES', 'TOTAL_VISUALIZACOES']
]
# df_videos_populares_categoria = df_videos_populares_categoria.query('ID_CATEGORIA == "20" and DATA_EXTRACAO == "2023-10-15"')

In [1274]:
df_teste = df_videos_populares_categoria.groupby(['DATA_EXTRACAO','ID_CATEGORIA', 'ID_VIDEO', 'TITULO_VIDEO']).agg(
    TOTAL_VISUALIZACOES=('TOTAL_VISUALIZACOES', 'max'),
    TOTAL_COMENTARIOS=('TOTAL_COMENTARIOS', 'max'),
    TOTAL_LIKES=('TOTAL_LIKES', 'max')
).reset_index().sort_values(by= ['ID_VIDEO', 'DATA_EXTRACAO'])
df_teste.head(50)

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO  \
93      2023-10-15           20  -8A4gKs9CxE   
315     2023-10-16           20  -8A4gKs9CxE   
545     2023-10-19           20  -8A4gKs9CxE   
776     2023-10-20           20  -8A4gKs9CxE   
993     2023-10-21           20  -8A4gKs9CxE   
1223    2023-10-22           20  -8A4gKs9CxE   
606     2023-10-19           24  -BXItmWzWNE   
831     2023-10-20           24  -BXItmWzWNE   
1051    2023-10-21           24  -BXItmWzWNE   
1290    2023-10-22           24  -BXItmWzWNE   
1526    2023-10-23           24  -BXItmWzWNE   
1761    2023-10-24           24  -BXItmWzWNE   
1993    2023-10-25           24  -BXItmWzWNE   
2241    2023-10-26           24  -BXItmWzWNE   
2487    2023-10-27           24  -BXItmWzWNE   
2320    2023-10-27           10  -DkJMa7QkGY   
53      2023-10-15           17  -FGiCPBn0oI   
273     2023-10-16           17  -FGiCPBn0oI   
507     2023-10-19           17  -FGiCPBn0oI   
741     2023-10-20           17  -FGiCPBn0oI   
54      2023-10-15           17  -dOR48RgOvc   
274     2023-10-16           17  -dOR48RgOvc   
1994    2023-10-25           24  -rqDYATxG4Y   
2242    2023-10-26           24  -rqDYATxG4Y   
2488    2023-10-27           24  -rqDYATxG4Y   
990     2023-10-21           19  05yeNSE8jDc   
1219    2023-10-22           19  05yeNSE8jDc   
1220    2023-10-22           19  05yeNSE8jDc   
1456    2023-10-23           19  05yeNSE8jDc   
1690    2023-10-24           19  05yeNSE8jDc   
1928    2023-10-25           19  05yeNSE8jDc   
2168    2023-10-26           19  05yeNSE8jDc   
2418    2023-10-27           19  05yeNSE8jDc   
1185    2023-10-22           17  0Gq4kqTz3NE   
1416    2023-10-23           17  0Gq4kqTz3NE   
1653    2023-10-24           17  0Gq4kqTz3NE   
1883    2023-10-25           17  0Gq4kqTz3NE   
2112    2023-10-26           17  0Gq4kqTz3NE   
55      2023-10-15           17  0US496Tu1U4   
275     2023-10-16           17  0US496Tu1U4   
508     2023-10-19           17  0US496Tu1U4   
742     2023-10-20           17  0US496Tu1U4   
964     2023-10-21           17  0US496Tu1U4   
1186    2023-10-22           17  0US496Tu1U4   
94      2023-10-15           20  0Z-jJ1ySshY   
316     2023-10-16           20  0Z-jJ1ySshY   
546     2023-10-19           20  0Z-jJ1ySshY   
777     2023-10-20           20  0Z-jJ1ySshY   
158     2023-10-15           24  0Z86ZCnSA3w   
924     2023-10-21           10  0cVDA5eGyew   

                                           TITULO_VIDEO  TOTAL_VISUALIZACOES  \
93                      ENTREI NO CÉREBRO DO MEU AMIGO!               133464   
315                     ENTREI NO CÉREBRO DO MEU AMIGO!               161525   
545                     ENTREI NO CÉREBRO DO MEU AMIGO!               284886   
776                     ENTREI NO CÉREBRO DO MEU AMIGO!               346730   
993                     ENTREI NO CÉREBRO DO MEU AMIGO!               452444   
1223                    ENTREI NO CÉREBRO DO MEU AMIGO!               544631   
606                                   skibidi toilet 66             10476171   
831                                   skibidi toilet 66             29685000   
1051                                  skibidi toilet 66             39102892   
1290                                  skibidi toilet 66             44361844   
1526                                  skibidi toilet 66             47972664   
1761                                  skibidi toilet 66             51071484   
1993                                  skibidi toilet 66             53698760   
2241                                  skibidi toilet 66             56102804   
2487                                  skibidi toilet 66             58424408   
2320  Tyla - Water (Live from The Tonight Show Starr...               403054   
53    🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...                72069   
273   🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...                75882   
507   🎥 BASTIDORES | FORTALEZA X COR

In [1275]:
df_teste['TOTAL_VISUALIZACOES_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_VISUALIZACOES'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_VISUALIZACOES_DIA_ATUAL'] = df_teste['TOTAL_VISUALIZACOES'] - df_teste['TOTAL_VISUALIZACOES_DESLOCADO']

df_teste['TOTAL_COMENTARIOS_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_COMENTARIOS'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_COMENTARIOS_DIA_ATUAL'] = df_teste['TOTAL_COMENTARIOS'] - df_teste['TOTAL_COMENTARIOS_DESLOCADO']

df_teste['TOTAL_LIKES_DESLOCADO'] = df_teste.groupby(['ID_VIDEO'])['TOTAL_LIKES'].shift(1)
df_teste.fillna(0, axis=1, inplace=True)
df_teste['TOTAL_LIKES_DIA_ATUAL'] = df_teste['TOTAL_LIKES'] - df_teste['TOTAL_LIKES_DESLOCADO']

In [1276]:
df_teste.drop(['TOTAL_COMENTARIOS_DESLOCADO', 'TOTAL_COMENTARIOS', 'TOTAL_VISUALIZACOES_DESLOCADO', 'TOTAL_VISUALIZACOES', 'TOTAL_LIKES_DESLOCADO', 'TOTAL_LIKES'], axis=1, inplace=True)
df_teste.head(45)

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO  \
93    2023-10-15 00:00:00           20  -8A4gKs9CxE   
315   2023-10-16 00:00:00           20  -8A4gKs9CxE   
545   2023-10-19 00:00:00           20  -8A4gKs9CxE   
776   2023-10-20 00:00:00           20  -8A4gKs9CxE   
993   2023-10-21 00:00:00           20  -8A4gKs9CxE   
1223  2023-10-22 00:00:00           20  -8A4gKs9CxE   
606   2023-10-19 00:00:00           24  -BXItmWzWNE   
831   2023-10-20 00:00:00           24  -BXItmWzWNE   
1051  2023-10-21 00:00:00           24  -BXItmWzWNE   
1290  2023-10-22 00:00:00           24  -BXItmWzWNE   
1526  2023-10-23 00:00:00           24  -BXItmWzWNE   
1761  2023-10-24 00:00:00           24  -BXItmWzWNE   
1993  2023-10-25 00:00:00           24  -BXItmWzWNE   
2241  2023-10-26 00:00:00           24  -BXItmWzWNE   
2487  2023-10-27 00:00:00           24  -BXItmWzWNE   
2320  2023-10-27 00:00:00           10  -DkJMa7QkGY   
53    2023-10-15 00:00:00           17  -FGiCPBn0oI   
273   2023-10-16 00:00:00           17  -FGiCPBn0oI   
507   2023-10-19 00:00:00           17  -FGiCPBn0oI   
741   2023-10-20 00:00:00           17  -FGiCPBn0oI   
54    2023-10-15 00:00:00           17  -dOR48RgOvc   
274   2023-10-16 00:00:00           17  -dOR48RgOvc   
1994  2023-10-25 00:00:00           24  -rqDYATxG4Y   
2242  2023-10-26 00:00:00           24  -rqDYATxG4Y   
2488  2023-10-27 00:00:00           24  -rqDYATxG4Y   
990   2023-10-21 00:00:00           19  05yeNSE8jDc   
1219  2023-10-22 00:00:00           19  05yeNSE8jDc   
1220  2023-10-22 00:00:00           19  05yeNSE8jDc   
1456  2023-10-23 00:00:00           19  05yeNSE8jDc   
1690  2023-10-24 00:00:00           19  05yeNSE8jDc   
1928  2023-10-25 00:00:00           19  05yeNSE8jDc   
2168  2023-10-26 00:00:00           19  05yeNSE8jDc   
2418  2023-10-27 00:00:00           19  05yeNSE8jDc   
1185  2023-10-22 00:00:00           17  0Gq4kqTz3NE   
1416  2023-10-23 00:00:00           17  0Gq4kqTz3NE   
1653  2023-10-24 00:00:00           17  0Gq4kqTz3NE   
1883  2023-10-25 00:00:00           17  0Gq4kqTz3NE   
2112  2023-10-26 00:00:00           17  0Gq4kqTz3NE   
55    2023-10-15 00:00:00           17  0US496Tu1U4   
275   2023-10-16 00:00:00           17  0US496Tu1U4   
508   2023-10-19 00:00:00           17  0US496Tu1U4   
742   2023-10-20 00:00:00           17  0US496Tu1U4   
964   2023-10-21 00:00:00           17  0US496Tu1U4   
1186  2023-10-22 00:00:00           17  0US496Tu1U4   
94    2023-10-15 00:00:00           20  0Z-jJ1ySshY   

                                           TITULO_VIDEO  \
93                      ENTREI NO CÉREBRO DO MEU AMIGO!   
315                     ENTREI NO CÉREBRO DO MEU AMIGO!   
545                     ENTREI NO CÉREBRO DO MEU AMIGO!   
776                     ENTREI NO CÉREBRO DO MEU AMIGO!   
993                     ENTREI NO CÉREBRO DO MEU AMIGO!   
1223                    ENTREI NO CÉREBRO DO MEU AMIGO!   
606                                   skibidi toilet 66   
831                                   skibidi toilet 66   
1051                                  skibidi toilet 66   
1290                                  skibidi toilet 66   
1526                                  skibidi toilet 66   
1761                                  skibidi toilet 66   
1993                                  skibidi toilet 66   
2241                                  skibidi toilet 66   
2487                                  skibidi toilet 66   
2320  Tyla - Water (Live from The Tonight Show Starr...   
53    🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...   
273   🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...   
507   🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...   
741   🎥 BASTIDORES | FORTALEZA X CORINTHIANS | CONME...   
54    TITE NO FLA! DEPOIS DE MUITOS ERROS, O FLAMENG...   
274   TITE NO FLA! DEPOIS DE MUITOS ERROS, O FLAMENG...   
1994  Simioni sobra no Resta Um e ocupa a última vag...   
2242  Simioni sobra no Resta Um e ocupa a última vag...   
2488  

In [1280]:
df_teste = pd.merge(df_teste, lista_categoria, left_on=['ID_CATEGORIA'], right_on=['ID'], how='inner')
df_teste

DATA_EXTRACAO ID_CATEGORIA     ID_VIDEO                     TITULO_VIDEO  \
0       2023-10-15           20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
1       2023-10-16           20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
2       2023-10-19           20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
3       2023-10-20           20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
4       2023-10-21           20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
...            ...          ...          ...                              ...   
2552    2023-10-23           26  ua1DsYjuavo                TOUR NO MEU SÍTIO   
2553    2023-10-24           26  ua1DsYjuavo                TOUR NO MEU SÍTIO   
2554    2023-10-25           26  ua1DsYjuavo                TOUR NO MEU SÍTIO   
2555    2023-10-26           26  ua1DsYjuavo                TOUR NO MEU SÍTIO   
2556    2023-10-27           26  ua1DsYjuavo                TOUR NO MEU SÍTIO   

      TOTAL_VISUALIZACOES_DIA_ATUAL  TOTAL_COMENTARIOS_DIA_ATUAL  \
0                            133464                          168   
1                             28061                            6   
2                            123361                           13   
3                             61844                           10   
4                            105714                           12   
...                             ...                          ...   
2552                          38867                          314   
2553                          16162                          105   
2554                          26965                          274   
2555                          35736                          303   
2556                          29960                          249   

      TOTAL_LIKES_DIA_ATUAL  ID NOME_CATEGORIA  
0                      5392  20         Gaming  
1                       781  20         Gaming  
2                      2360  20         Gaming  
3                      1257  20         Gaming  
4                      2046  20         Gaming  
...                     ...  ..            ...  
2552                   2563  26  Howto & Style  
2553                    995  26  Howto & Style  
2554                   2406  26  Howto & Style  
2555                   2814  26  Howto & Style  
2556                   2221  26  Howto & Style  

[2557 rows x 9 columns]

In [1281]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   DATA_EXTRACAO                  2557 non-null   datetime64[ns]
 1   ID_CATEGORIA                   2557 non-null   object        
 2   ID_VIDEO                       2557 non-null   string        
 3   TITULO_VIDEO                   2557 non-null   string        
 4   TOTAL_VISUALIZACOES_DIA_ATUAL  2557 non-null   int32         
 5   TOTAL_COMENTARIOS_DIA_ATUAL    2557 non-null   int32         
 6   TOTAL_LIKES_DIA_ATUAL          2557 non-null   int32         
 7   ID                             2557 non-null   object        
 8   NOME_CATEGORIA                 2557 non-null   object        
dtypes: datetime64[ns](1), int32(3), object(3), string(2)
memory usage: 149.9+ KB


In [1282]:
df_teste.drop('ID', axis=1, inplace=True)

In [1286]:
df_teste['DATA_EXTRACAO'] = df_teste['DATA_EXTRACAO'].astype('datetime64[ns]')
df_teste[['ID_VIDEO', 'TITULO_VIDEO']] = df_teste[['ID_VIDEO', 'TITULO_VIDEO']].astype('string')
df_teste[['TOTAL_VISUALIZACOES_DIA_ATUAL', 'TOTAL_COMENTARIOS_DIA_ATUAL', 'TOTAL_LIKES_DIA_ATUAL']] = df_teste[['TOTAL_VISUALIZACOES_DIA_ATUAL', 'TOTAL_COMENTARIOS_DIA_ATUAL', 'TOTAL_LIKES_DIA_ATUAL']].astype('int32')
df_teste['NOME_CATEGORIA'] = df_teste['NOME_CATEGORIA'].astype('string')
df_teste['ID_CATEGORIA'] = df_teste['ID_CATEGORIA'].astype('int32')
df_teste

DATA_EXTRACAO  ID_CATEGORIA     ID_VIDEO  \
0       2023-10-15            20  -8A4gKs9CxE   
1       2023-10-16            20  -8A4gKs9CxE   
2       2023-10-19            20  -8A4gKs9CxE   
3       2023-10-20            20  -8A4gKs9CxE   
4       2023-10-21            20  -8A4gKs9CxE   
...            ...           ...          ...   
2552    2023-10-23            26  ua1DsYjuavo   
2553    2023-10-24            26  ua1DsYjuavo   
2554    2023-10-25            26  ua1DsYjuavo   
2555    2023-10-26            26  ua1DsYjuavo   
2556    2023-10-27            26  ua1DsYjuavo   

                         TITULO_VIDEO  TOTAL_VISUALIZACOES_DIA_ATUAL  \
0     ENTREI NO CÉREBRO DO MEU AMIGO!                         133464   
1     ENTREI NO CÉREBRO DO MEU AMIGO!                          28061   
2     ENTREI NO CÉREBRO DO MEU AMIGO!                         123361   
3     ENTREI NO CÉREBRO DO MEU AMIGO!                          61844   
4     ENTREI NO CÉREBRO DO MEU AMIGO!                         105714   
...                               ...                            ...   
2552                TOUR NO MEU SÍTIO                          38867   
2553                TOUR NO MEU SÍTIO                          16162   
2554                TOUR NO MEU SÍTIO                          26965   
2555                TOUR NO MEU SÍTIO                          35736   
2556                TOUR NO MEU SÍTIO                          29960   

      TOTAL_COMENTARIOS_DIA_ATUAL  TOTAL_LIKES_DIA_ATUAL NOME_CATEGORIA  
0                             168                   5392         Gaming  
1                               6                    781         Gaming  
2                              13                   2360         Gaming  
3                              10                   1257         Gaming  
4                              12                   2046         Gaming  
...                           ...                    ...            ...  
2552                          314                   2563  Howto & Style  
2553                          105                    995  Howto & Style  
2554                          274                   2406  Howto & Style  
2555                          303                   2814  Howto & Style  
2556                          249                   2221  Howto & Style  

[2557 rows x 8 columns]

In [1287]:
df_teste.head()

DATA_EXTRACAO  ID_CATEGORIA     ID_VIDEO                     TITULO_VIDEO  \
0    2023-10-15            20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
1    2023-10-16            20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
2    2023-10-19            20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
3    2023-10-20            20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   
4    2023-10-21            20  -8A4gKs9CxE  ENTREI NO CÉREBRO DO MEU AMIGO!   

   TOTAL_VISUALIZACOES_DIA_ATUAL  TOTAL_COMENTARIOS_DIA_ATUAL  \
0                         133464                          168   
1                          28061                            6   
2                         123361                           13   
3                          61844                           10   
4                         105714                           12   

   TOTAL_LIKES_DIA_ATUAL NOME_CATEGORIA  
0                   5392         Gaming  
1                    781         Gaming  
2                   2360         Gaming  
3                   1257         Gaming  
4                   2046         Gaming

In [1288]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2557 entries, 0 to 2556
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   DATA_EXTRACAO                  2557 non-null   datetime64[ns]
 1   ID_CATEGORIA                   2557 non-null   int32         
 2   ID_VIDEO                       2557 non-null   string        
 3   TITULO_VIDEO                   2557 non-null   string        
 4   TOTAL_VISUALIZACOES_DIA_ATUAL  2557 non-null   int32         
 5   TOTAL_COMENTARIOS_DIA_ATUAL    2557 non-null   int32         
 6   TOTAL_LIKES_DIA_ATUAL          2557 non-null   int32         
 7   NOME_CATEGORIA                 2557 non-null   string        
dtypes: datetime64[ns](1), int32(4), string(3)
memory usage: 120.0 KB


- Desempenho da categoria dia a dia